In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import gc
import os
import re
from sklearn.decomposition import PCA

In [ ]:
path = 'E:/practice coding/kaggle/M5 Forecasting - Accuracy/m5-forecasting-accuracy/'

In [ ]:
#function to reduce memory used
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

**1. DATA PREPARATION**

In [ ]:
#Read data files
sales_train_validation = pd.read_csv(path + 'sales_train_validation.csv')
sell_price = pd.read_csv(path + 'sell_prices.csv')
calendar = pd.read_csv(path + 'calendar.csv')

In [ ]:
#convert sales_train_valdation table to have a day column and a demand column
sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
#convert day column from string to number of days in integer
sales_train_validation['day'] = sales_train_validation['day'].apply(lambda x: int(x[2:])).astype('int16')
#Because data is too large, days larger or equal to 1434 are only selected for the analysis
sales_train_validation = sales_train_validation.loc[sales_train_validation.day >= 1434] #day 1434 is 01/01/2015

Analyze Canlendar Data

In [ ]:
#convert day column from string to number of days in integer
calendar['d'] = calendar['d'].apply(lambda x: int(x[2:])).astype('int16')

In [ ]:
#Function to add more day and time features
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [ ]:
#Because the add_datepart function already has Dayofweek column, weekday column of calendar will be dropped to save memory
calendar.drop(['weekday'], axis=1, inplace=True)

In [ ]:
#Add day and time features to calendar
add_datepart(calendar, 'date', drop=False)

In [ ]:
#Function to add season feature
def to_season(date):    
    #convert date to season for the northen hemisphere
    #"day of year" ranges for the northern hemisphere
    spring = range(80, 172)
    summer = range(172, 264)
    fall = range(264, 355)
    # winter = everything else
    doy = date.timetuple().tm_yday
    if doy in spring:
      season = 'Spring'
    elif doy in summer:
      season = 'Summer'
    elif doy in fall:
      season = 'Fall'
    else:
      season = 'Winter'
    return season

In [ ]:
#Add season feature to calendar
calendar['Season'] = calendar.date.transform(lambda x: to_season(x))

Merge Calendar Data and Sell Price Data with Sales Train Validation Data

In [ ]:
sales_train_validation = sales_train_validation.merge(calendar, how='left', left_on='day', right_on='d')
sales_train_validation.drop('d', axis=1, inplace=True)
sales_train_validation = sales_train_validation.merge(sell_price, how='left', on=['item_id', 'store_id', 'wm_yr_wk'])

**2. PREPARE FOR PREDICTION DATA**

Because the lag features and rolling window will be created for the analysis. The prediction data will be added to the training data to make lag features and rolling window for them too

In [ ]:
#Read the prediction data
sample_submission = pd.read_csv(path+'sample_submission.csv')

#Because the sale train validation data was melted so it was changed. To get the columns needed for prediction data
#the original sales train validation data is read again
sales_train_validation1 = pd.read_csv(path + 'sales_train_validation.csv')
sales_train_validation1 = sales_train_validation1[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']]

#Only take the validation rows which is 28 days after the trainning data day for the prediction
validation = sample_submission.loc[sample_submission.id.str.contains('validation')]
#Get id, item_id, dept_id, cat_id, store_id, state_id for the validation data for prediction
validation = validation.merge(sales_train_validation1, how='left', on='id')

validation = pd.melt(validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
validation['day'] = validation['day'].apply(lambda x: int(x[1:])).astype('int16')
#The training data day is end at 1913, so the day column of validation is added to 1913 to consider the days after
#the trainning data day
validation['day'] = validation['day'] + 1913
validation = validation.merge(calendar, how='left', left_on='day', right_on='d')
validation.drop('d', axis=1, inplace=True)
validation = validation.merge(sell_price, how='left', on=['item_id', 'store_id', 'wm_yr_wk',])

**3. PREPARING FEATURES**

In [ ]:
data = pd.concat((sales_train_validation, validation)

* Fill missing data for sell price

In [ ]:
def fillna_price(data):
  mean_sell_price = data.loc[data.sell_price.isnull()==False][['item_id','sell_price']].groupby('item_id', as_index=False).mean()
  mean_sell_price_per_item = {}
  for i in range(mean_sell_price.shape[0]):
    mean_sell_price_per_item[mean_sell_price['item_id'].iloc[i]] = mean_sell_price['sell_price'].iloc[i]
    
  sell_price_nan = data['sell_price'].isnull().values
  sell_price_nan_loc = np.where(sell_price_nan == True)[0]

  for i in sell_price_nan_loc:
    id_ = data['item_id'].iloc[i]
    data['sell_price'].iloc[i] = mean_sell_price_per_item[id_]

  return data

In [ ]:
data = fillna_price(data)

* Create more features

In [ ]:
def create_fe(data):
    
    shift_window_size = [28]
    lag_day = [1,2,3,4,5,6,7]
    roll_demand_days = [7,14,30,60,180]
    
    #Add lag demand feature     
    for i in shift_window_size:
        for j in lag_day:
            data['lag_pos'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i+j))
       

    ### Moving average on item_id         
    for i in shift_window_size:
      for j in roll_demand_days:
        data['demand_roll_mean_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(window=j, min_periods=1).mean())
        data['demand_roll_std_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(window=j, min_periods=1).std())        
        data['demand_roll_q25_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(window=j, min_periods=1).quantile(0.25))        
        data['demand_roll_q75_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(window=j, min_periods=1).quantile(0.75))        
        data['demand_roll_IQR_'+str(j)] = data['demand_roll_q75_'+str(j)] - data['demand_roll_q25_'+str(j)]    
        
    
    ### Moving average on item_id and dept_id
    for i in shift_window_size:
       for j in roll_demand_days:
         data['demand_dept_roll_mean_'+str(j)] = data.groupby(['item_id','dept_id'], as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(j).mean())
    
    return data

In [ ]:
data = create_fe(data)

In [ ]:
#reduce memory used by data
data = reduce_mem_usage(data)
gc.collect()

**4. XGBOOST MODEL**

In [ ]:
from xgboost import XGBRegressor as xgb

In [ ]:
X = data.loc[data.day <= 1913].drop(['id','demand','date'], axis=1)
y = data.loc[data.day <= 1913]['demand']

In [ ]:
xgb_model = xgb(n_estimators=1000, learning_rate=0.03, n_jobs=-1, colsample_bytree=1.0,
                eta=0.01, max_depth=10, min_child_weight=6, objective='reg:linear',gamma=0.3,tree_method='exact')
xgb_model.fit(X, y, eval_set=[(X,y)], early_stopping_rounds=30, verbose=2)

**5. MAKE PREDICTION FOR 28 DAYS AFTER DAY 1913**

In [ ]:
validation_demand_pred = xgb_model.predict(data.loc[data.day > 1913].drop(['id','demand','date'], axis=1))